In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [33]:
#import tensorflow_hub as hub
import pandas as pd
import numpy as np
import chess
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import os



# Diviser le DataFrame en sous-ensembles
def parallelize_dataframe(df, func, num_processes=None):
    num_processes = num_processes or os.cpu_count()
    df_split = np.array_split(df, num_processes)
    with ProcessPoolExecutor(max_workers=num_processes) as executor:
        results = list(tqdm(executor.map(func, df_split), total=len(df_split), desc="Processing"))
    return pd.concat(results)




def boardstate(fen):
    board = chess.Board(fen[0])
    fstr = str(fen[0])

    if board.has_kingside_castling_rights(chess.WHITE) == True:
        WCKI = 1
    else:
        WCKI = 0
    if board.has_queenside_castling_rights(chess.WHITE) == True:
        WCQ = 1
    else:
        WCQ = 0
    if board.is_check() == True:
        WCH = 1
    else:
        WCH = 0

    if board.has_kingside_castling_rights(chess.BLACK) == True:
        BCKI = 1
    else:
        BCKI = 0
    if board.has_queenside_castling_rights(chess.BLACK) == True:
        BCQ = 1
    else:
        BCQ = 0
    if board.was_into_check() == True:
        BCH = 1
    else:
        BCH = 0

    #f = [M, WCKI, WCQ, WCH, BCKI, BCQ, BCH]
    fw = [WCKI, WCQ, WCH]
    fb = [BCKI, BCQ, BCH]

    bstr = str(board)
    bstr = bstr.replace("p", "\ -1")
    bstr = bstr.replace("n", "\ -3")
    bstr = bstr.replace("b", "\ -4")
    bstr = bstr.replace("r", "\ -5")
    bstr = bstr.replace("q", "\ -9")
    bstr = bstr.replace("k", "\ -100")
    bstr = bstr.replace("P", "\ 1")
    bstr = bstr.replace("N", "\ 3")
    bstr = bstr.replace("B", "\ 4")
    bstr = bstr.replace("R", "\ 5")
    bstr = bstr.replace("Q", "\ 9")
    bstr = bstr.replace("K", "\ 100")
    bstr = bstr.replace(".", "\ 0")
    bstr = bstr.replace("\ ", ",")
    bstr = bstr.replace("'", " ")
    bstr = bstr.replace("\n", "")
    bstr = bstr.replace(" ", "")
    bstr = bstr[1:]
    bstr = eval(bstr)
    bstr = list(bstr)
    if "w" not in fstr:
        for i in range(len(bstr)):
            bstr[i] = bstr[i] * -1
        bstr.reverse()
        fs = fb
        fb = fw
        fw = fs


    BITBOARD = fw + fb + bstr


    return BITBOARD
#x = ''
#r = 1
def strfix(fen, tr):
    
    fstr = str(fen)
    
    if '#' in str(tr):
        if '-' in tr:
            t = -32000
        else:
            t = 32000
    elif '\ufeff+23' in str(tr):
        t = 0
    else:
        t = int(tr)
        
    # check if this actually helps hurts or even makes sense 
    
    # check if this actually helps hurts or even makes sense 
    
    # check if this actually helps hurts or even makes sense 
    
    # by this i mean fliping every black evaluation
    if "w" not in fstr:
        t = t*-1

    
    
    t = t/10
    
    t = min(3200, t)
    t = max(-3200, t)
    #try:
    #    t = int(t)
    #except ValueError:
    #    t = 0

    return t
#print(x)

data = pd.read_csv('../dataset/chessData.csv')

# print(data)

label_columns = [1]

data_features = data.drop(columns=data.iloc[:, label_columns])
# data_features = data_features.iloc[600000:2000000]
data_features = data_features.head(5000000)

# cut out the line below apply the fen check and then cut the collumn containing the fens because right now the evaluation flips are still completely missaligned

# data_labels = data.iloc[:, label_columns]

data_labels = data

data_labels.columns = ['col1', 'col2']



data_labels = data_labels.head(5000000)
# data_labels = data_labels.iloc[600000:2000000]

data_labels = data_labels.astype(str) 

# Diviser en morceaux pour multiprocessing
def process_chunk(chunk):
    return chunk.apply(lambda x: strfix(x['col1'], x['col2']), axis=1)
tqdm.pandas(desc="Processing labels")
data_labels = parallelize_dataframe(data_labels, process_chunk)


#print(data_features)

print(data_labels)

#print(data_labels.dtypes)

def process_chunk(chunk):
    return chunk.apply(boardstate, axis=1)
tqdm.pandas(desc="Processing features 1/2")
data_features = parallelize_dataframe(data_features, process_chunk)

def process_chunk(chunk):
    return chunk.apply(pd.Series)
tqdm.pandas(desc="Processing features 2/2")
data_features = parallelize_dataframe(data_features, process_chunk)

#print(data_features)

input2_columns = [0, 1, 2, 3, 4, 5]


inputboard = data_features.drop(columns=data_features.iloc[:, input2_columns]) 

#print(inputboard.dtypes)

inputboard = np.array(inputboard)
#print(inputboard.shape)

print(inputboard)


inputmeta = data_features.iloc[:, input2_columns]

#print(inputmeta.dtypes)

inputmeta = np.array(inputmeta)
#print(inputmeta.shape)

#print(inputmeta)


<>:57: SyntaxWarning: invalid escape sequence '\ '
<>:58: SyntaxWarning: invalid escape sequence '\ '
<>:59: SyntaxWarning: invalid escape sequence '\ '
<>:60: SyntaxWarning: invalid escape sequence '\ '
<>:61: SyntaxWarning: invalid escape sequence '\ '
<>:62: SyntaxWarning: invalid escape sequence '\ '
<>:63: SyntaxWarning: invalid escape sequence '\ '
<>:64: SyntaxWarning: invalid escape sequence '\ '
<>:65: SyntaxWarning: invalid escape sequence '\ '
<>:66: SyntaxWarning: invalid escape sequence '\ '
<>:67: SyntaxWarning: invalid escape sequence '\ '
<>:68: SyntaxWarning: invalid escape sequence '\ '
<>:69: SyntaxWarning: invalid escape sequence '\ '
<>:70: SyntaxWarning: invalid escape sequence '\ '
<>:57: SyntaxWarning: invalid escape sequence '\ '
<>:58: SyntaxWarning: invalid escape sequence '\ '
<>:59: SyntaxWarning: invalid escape sequence '\ '
<>:60: SyntaxWarning: invalid escape sequence '\ '
<>:61: SyntaxWarning: invalid escape sequence '\ '
<>:62: SyntaxWarning: invalid e

0           1.0
1           5.6
2           0.9
3           5.2
4           2.6
           ... 
4999995    -0.4
4999996    13.0
4999997    -0.5
4999998     6.2
4999999    -4.2
Length: 5000000, dtype: float64


/Data/augustin.kheng/chessAI/venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
Processing:   0%|          | 0/16 [00:00<?, ?it/s]/tmp/ipykernel_3292839/2057185092.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  board = chess.Board(fen[0])
/tmp/ipykernel_3292839/2057185092.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fstr = str(fen[0])
/tmp/ipykernel_3292839/2057185092.py:23: FutureWarning: Series.__getitem__ treating keys as posi

[[-5 -3 -4 ...  4  3  5]
 [-5 -3 -4 ...  4  3  5]
 [-5 -3 -4 ...  4  3  5]
 ...
 [-5  0  0 ...  0  0  5]
 [-5  0  0 ...  0  0  5]
 [-5  0  0 ...  0  0  5]]


In [35]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dense, Dropout
from tensorflow.keras.saving import register_keras_serializable

# Déclarez la classe EncoderLayer comme sérialisable
@register_keras_serializable(package="Custom", name="EncoderLayer")
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        super(EncoderLayer, self).__init__(**kwargs)
        self.output_dim = output_dim
        self.dense1 = Dense(16, activation='relu')
        self.dense2 = Dense(output_dim, activation='relu')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return x

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dense2.units)

    def get_config(self):
        config = super(EncoderLayer, self).get_config()
        config.update({"output_dim": self.output_dim})
        return config

# Inputs
input1 = tf.keras.layers.Input(shape=(64,))
input2 = tf.keras.layers.Input(shape=(6,))

# Reshape input1
reshaped_input1 = tf.keras.layers.Reshape(target_shape=(64, 1))(input1)

# Encoder layer
encoder_layer = EncoderLayer(output_dim=8)
encoded_input1 = tf.keras.layers.TimeDistributed(encoder_layer)(reshaped_input1)

# Transformer block function
def transformer_block(x, num_heads, ff_dim, dropout_rate=0.1):
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=x.shape[-1])(x, x)
    attn_output = Dropout(dropout_rate)(attn_output)
    attn_output = LayerNormalization(epsilon=1e-6)(x + attn_output)

    ff_output = Dense(ff_dim, activation='relu')(attn_output)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Dense(x.shape[-1])(ff_output)
    return LayerNormalization(epsilon=1e-6)(attn_output + ff_output)

# Apply 3 transformer blocks
x = encoded_input1
for _ in range(3):
    x = transformer_block(x, num_heads=4, ff_dim=64, dropout_rate=0.1)

# Flatten transformer output
flattened_transformer = tf.keras.layers.Flatten()(x)

# Concatenate with input2
conc = tf.keras.layers.concatenate([flattened_transformer, input2])

# Dense layers
Denselayer1 = tf.keras.layers.Dense(1024, activation='relu')(conc)
Denselayer2 = tf.keras.layers.Dense(512, activation='relu')(Denselayer1)
Denselayer3 = tf.keras.layers.Dense(256, activation='relu')(Denselayer2)
Denselayer4 = tf.keras.layers.Dense(256, activation='relu')(Denselayer3)
Output = tf.keras.layers.Dense(1, activation='linear')(Denselayer4)

# Define and compile model
data_model = tf.keras.models.Model(inputs=[input1, input2], outputs=Output)

# Charger un modèle sauvegardé avec les couches personnalisées
try:
    data_model = load_model(
        "engine_with_transformers.keras",
        custom_objects={"EncoderLayer": EncoderLayer}
    )
except Exception as e:
    print(f"Erreur lors du chargement du modèle : {e}")

metric = [tf.keras.metrics.MeanAbsoluteError()]
opt = tf.keras.optimizers.Adam()
los = tf.keras.losses.MeanSquaredError()

data_model.compile(optimizer=opt, 
                   loss=los,
                   metrics=metric)

data_model.summary()

# Train and save the model
for i in range(10):
    data_model.fit([inputboard, inputmeta], data_labels, epochs=1, batch_size=8192, shuffle=True)
    data_model.save(f"./engine_with_transformers_epoch_{i+1}.keras")


/Data/augustin.kheng/chessAI/venv/lib/python3.12/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'encoder_layer_7', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/Data/augustin.kheng/chessAI/venv/lib/python3.12/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'encoder_layer_3', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Erreur lors du chargement du modèle : A total of 2 objects could not be loaded. Example error message for object <Dense name=dense_108, built=False>:

Layer 'dense_108' was never built and thus it doesn't have any variables. However the weights file lists 2 variables for this layer.
In most cases, this error indicates that either:

1. The layer is owned by a parent layer that implements a `build()` method, but calling the parent's `build()` method did NOT create the state of the child layer 'dense_108'. A `build()` method must create ALL state for the layer, including the state of any children layers.

2. You need to implement the `def build_from_config(self, config)` method on layer 'dense_108', to specify how to rebuild it during loading. In this case, you might also want to implement the method that generates the build config at saving time, `def get_build_config(self)`. The method `build_from_config()` is meant to create the state of the layer (i.e. its variables) upon deserializat

/Data/augustin.kheng/chessAI/venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 118 variables whereas the saved optimizer has 126 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, 64)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_12          │ (None, 64, 1)     │          0 │ input_layer_14[0… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_7  │ (None, 64, 8)     │          0 │ reshape_12[0][0]  │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 64, 8)     │      1,128 │ time_distributed… │
│ (MultiHeadAttentio… │                   │            │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_70          │ (None, 64, 8)     │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_42 (Add)        │ (None, 64, 8)     │          0 │ time_distributed… │
│                     │                   │            │ dropout_70[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64, 8)     │         16 │ add_42[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_97 (Dense)    │ (None, 64, 64)    │        576 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_71          │ (None, 64, 64)    │          0 │ dense_97[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_98 (Dense)    │ (None, 64, 8)     │        520 │ dropout_71[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_43 (Add)        │ (None, 64, 8)     │          0 │ layer_normalizat… │
│                     │                   │            │ dense_98[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64, 8)     │         16 │ add_43[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 64, 8)     │      1,128 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_73          │ (None, 64, 8)     │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_44 (Add)        │ (None, 64, 8)     │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_73[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64, 8)     │         16 │ add_44[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_99 (Dense)    │ (None, 64, 64)    │        576 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_74          │ (None, 64, 64)    │          0 │ dense_99[0][0]  

 Total params: 1,260,401 (4.81 MB)

 Trainable params: 1,260,401 (4.81 MB)

 Non-trainable params: 0 (0.00 B)

611/611 ━━━━━━━━━━━━━━━━━━━━ 2751s 4s/step - loss: 129774.5391 - mean_absolute_error: 82.0384
611/611 ━━━━━━━━━━━━━━━━━━━━ 2745s 4s/step - loss: 116674.7734 - mean_absolute_error: 84.5917
245/611 ━━━━━━━━━━━━━━━━━━━━ 27:35 5s/step - loss: 111920.8438 - mean_absolute_error: 82.3765

KeyboardInterrupt: 